In [1]:
import pandas as pd
from dateutil.parser import parse

In [10]:
file = open("data/chat_data.txt", encoding = "utf8")

In [11]:
chats = file.readlines()

In [12]:
chats[:10]

['19/09/2021, 11:27 pm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n',
 '06/10/2019, 4:06 pm - +27 61 290 0591 created group "Young Data Professionals"\n',
 "19/09/2021, 11:27 pm - You joined using this group's invite link\n",
 '19/09/2021, 11:30 pm - +234 816 062 6971: 😂 that was fast\n',
 '19/09/2021, 11:31 pm - +234 818 368 5814: This one that people are leaving 😂😂😂\n',
 '19/09/2021, 11:31 pm - Ability: Mad o😂😂\n',
 '19/09/2021, 11:31 pm - +234 818 368 5814: Abhi they’ve seen money somewhere ni\n',
 '19/09/2021, 11:31 pm - Ability: People no dey sleep ni😂\n',
 '19/09/2021, 11:32 pm - Ability: Na another R guru enter this group low-key o👀\n',
 '19/09/2021, 11:32 pm - +234 816 062 6971: Nah only you no dey sleep\n']

In [13]:
def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy = fuzzy)
        return True
    except ValueError:
        return False

In [14]:
lines = []

for line in chats:
    line_list = line.replace("\n", "").split(",")
    
    if is_date(line_list[0]):
        lines.append([line_list[0], ("".join(line_list[1:]))])
        
    else:
        lines[-1][-1] = lines[-1][-1] + ' ' + line.replace("\n","")

In [15]:
tidy_format = pd.DataFrame(lines, columns = ['date','message'])

In [16]:
tidy_format.head()

,date,message
0,19/09/2021,11:27 pm - Messages and calls are end-to-end ...
1,06/10/2019,"4:06 pm - +27 61 290 0591 created group ""Youn..."
2,19/09/2021,11:27 pm - You joined using this group's invi...
3,19/09/2021,11:30 pm - +234 816 062 6971: 😂 that was fast
4,19/09/2021,11:31 pm - +234 818 368 5814: This one that p...


In [17]:
time_msg = tidy_format["message"].str.split("-", n = 1, expand = True)
tidy_format["time"] = time_msg[0]
tidy_format["message"] = time_msg[1]

In [ ]:
tidy_format.head()

In [18]:
user_msg = tidy_format["message"].str.split(":", n = 1, expand = True)
tidy_format["author"] = user_msg[0]
tidy_format["message"] = user_msg[1]

In [ ]:
tidy_format.head()

In [19]:
tidy_format['id'] = range(1, 1+len(tidy_format))

In [20]:
tidy_format = tidy_format[["id","date","time","author","message"]]

tidy_format.head()

,id,date,time,author,message
0,1,19/09/2021,11:27 pm,Messages and calls are end-to-end encrypted. ...,None
1,2,06/10/2019,4:06 pm,"+27 61 290 0591 created group ""Young Data Pro...",None
2,3,19/09/2021,11:27 pm,You joined using this group's invite link,None
3,4,19/09/2021,11:30 pm,+234 816 062 6971,😂 that was fast
4,5,19/09/2021,11:31 pm,+234 818 368 5814,This one that people are leaving 😂😂😂


In [21]:
pattern = r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)"

urls_extract = tidy_format.message.str.extractall(pattern)

urls_extract.head()

,,0
,match,
62,0,https://uk.indeed.com/rc/clk/dl?jk=c30cb4cea48...
63,0,https://uk.indeed.com/rc/clk/dl?jk=90161cdf743...
64,0,https://www.sli.do/careers/jobs-data-analyst
217,0,https://learning.edx.org/course/course-v1:MITx...
242,0,https://uk.indeed.com/m/viewjob?jk=f11e9a39f0e...


In [22]:
url_id = []

for i in range(0,len(urls_extract)):
    
    add_list = urls_extract.index[[i]][0][0] + 1
    
    url_id.append(add_list)

In [23]:
urls_extract['id'] = url_id

urls_extract.head()

,,0,id
,match,,
62,0,https://uk.indeed.com/rc/clk/dl?jk=c30cb4cea48...,63
63,0,https://uk.indeed.com/rc/clk/dl?jk=90161cdf743...,64
64,0,https://www.sli.do/careers/jobs-data-analyst,65
217,0,https://learning.edx.org/course/course-v1:MITx...,218
242,0,https://uk.indeed.com/m/viewjob?jk=f11e9a39f0e...,243


In [24]:
links_tbl = pd.merge(tidy_format, urls_extract, on = "id", how = "inner")

In [25]:
links_tbl.columns = ['id', 'date', 'time', 'author', 'message', 'url']

In [27]:
links_tbl.tail()

,id,date,time,author,message,url
466,17542,21/02/2022,5:26 pm,+234 813 780 3722,https://fabiosamedia.com/jobs/data-analyst/1206/,https://fabiosamedia.com/jobs/data-analyst/1206/
467,17555,21/02/2022,6:57 pm,+234 813 780 3722,https://toggl.com/jobs/toggltrack-data-analyst/,https://toggl.com/jobs/toggltrack-data-analyst/
468,17631,22/02/2022,12:27 am,+234 803 353 6901,https://z-lib.org/ Check here,https://z-lib.org/
469,17656,22/02/2022,12:30 pm,+234 703 806 5681,https://www.numbeo.com/cost-of-living/in/London,https://www.numbeo.com/cost-of-living/in/London
470,17672,22/02/2022,12:51 pm,+234 816 718 7102,https://www.linkedin.com/posts/piggyvest_pigg...,https://www.linkedin.com/posts/piggyvest_piggy...


In [28]:
links_tbl.shape

(471, 6)

In [ ]:
## links_tbl.to_csv("python_results.csv")

In [43]:
df["date_time"] = pd.to_datetime(df.Date + df.Time, errors = 'coerce')

In [35]:
tidy_format

,id,date,time,author,message,date_time
0,1,19/09/2021,11:27 pm,Messages and calls are end-to-end encrypted. ...,None,2021-09-19 23:27:00
1,2,06/10/2019,4:06 pm,"+27 61 290 0591 created group ""Young Data Pro...",None,2019-06-10 16:06:00
2,3,19/09/2021,11:27 pm,You joined using this group's invite link,None,2021-09-19 23:27:00
3,4,19/09/2021,11:30 pm,+234 816 062 6971,😂 that was fast,2021-09-19 23:30:00
4,5,19/09/2021,11:31 pm,+234 818 368 5814,This one that people are leaving 😂😂😂,2021-09-19 23:31:00
...,...,...,...,...,...,...
17679,17680,22/02/2022,1:19 pm,+234 708 531 6273,One thing I have come to find out is that the...,2022-02-22 13:19:00
17680,17681,22/02/2022,1:20 pm,+234 703 806 5681,The list gives you the standard cost of every...,2022-02-22 13:20:00
17681,17682,22/02/2022,1:20 pm,+234 816 381 5935,QESaid. I completely agree with you on this.,2022-02-22 13:20:00
17682,17683,22/02/2022,1:21 pm,+234 703 806 5681,London like Dublin has multiple zones. Same a...,2022-02-22 13:21:00


In [75]:
df.date_time.max()

Timestamp('2022-02-22 13:21:00')

In [63]:
last_record[0]

NameError: name 'last_record' is not defined

In [62]:
df.date_time.tail(1)

17673   2022-02-22 13:21:00
Name: date_time, dtype: datetime64[ns]

In [ ]:
if max(tidy_format.datetime) <= pd.to_datetime(last_record[0]):
            
            print("Records are upto date!\n\nCheck if you updated the chat data at data/chat_data.txt\n")

        else:

            new_records = tidy_format[tidy_format.datetime > last_record[0]]

In [45]:
dataset = chats[1:]

In [46]:

cleaned_data = []
for line in dataset:
    # Check, whether it is a new line or not
    # If the following characters are in the line -> assumption it is NOT a new line
    if '/' in line and ':' in line and ',' in line and '-' in line:
        # grab the info and cut it out
        date = line.split(",")[0]
        line2 = line[len(date):]
        time = line2.split("-")[0][2:]
        line3 = line2[len(time):]
        name = line3.split(":")[0][4:]
        line4 = line3[len(name):]
        message = line4[6:-1] # strip newline charactor
        cleaned_data.append([date, time, name, message])

    # else, assumption -> new line. Append new line to previous 'message'
    else:
        new = cleaned_data[-1][-1] + " " + line
        cleaned_data[-1][-1] = new

In [47]:
df = pd.DataFrame(cleaned_data, columns = ['Date', 'Time', 'Name', 'Message'])

In [74]:
df

,Date,Time,Name,Message,date_time
0,06/10/2019,4:06 pm,"+27 61 290 0591 created group ""Young Data Prof...",,2019-10-06 16:06:00
1,19/09/2021,11:27 pm,You joined using this group's invite link\n,,2021-09-19 23:27:00
2,19/09/2021,11:30 pm,+234 816 062 6971,😂 that was fast,2021-09-19 23:30:00
3,19/09/2021,11:31 pm,+234 818 368 5814,This one that people are leaving 😂😂😂,2021-09-19 23:31:00
4,19/09/2021,11:31 pm,Ability,Mad o😂😂,2021-09-19 23:31:00
...,...,...,...,...,...
17669,22/02/2022,1:19 pm,+234 708 531 6273,One thing I have come to find out is that thes...,2022-02-22 13:19:00
17670,22/02/2022,1:20 pm,+234 703 806 5681,The list gives you the standard cost of everyt...,2022-02-22 13:20:00
17671,22/02/2022,1:20 pm,+234 816 381 5935,QESaid. I completely agree with you on this.,2022-02-22 13:20:00
17672,22/02/2022,1:21 pm,+234 703 806 5681,"London, like Dublin, has multiple zones. Same ...",2022-02-22 13:21:00


In [51]:
df["date_time"] = df.Date + ' ' + df.Time , format="%d/%m/%Y %I:%M:%S %p"

In [73]:
df["date_time"] = pd.to_datetime(df.Date + ' ' + df.Time, errors = 'coerce', dayfirst=True)